# 필요한 라이브러리 불러오기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import joblib

import re
import json

from konlpy.tag import Mecab
import nltk

from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models import Word2Vec, Phrases

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [2]:
stop_words = "function,flashremoveCallback,오류,우회,하기,함수,추가,포토,기자,증권,뉴스,무단,배포,금지,전재 "
stop_words = stop_words.split(',')


In [3]:
stop_words = stop_words.split(',')

### 뉴스기사 전처리를 위한 함수

In [5]:
# 문서의 토큰화를 위해서 한국어만 남겨둠
def text_clean(doc):
    doc =re.sub("[^a-zA-Zㄱ-ㅎㅏ-ㅣ가-힣▲△↑▼▽↓ ]", "", doc)
    return doc

# 문서를 명사로 토큰화
def noun_tokenizer(text):
    nouns = Mecab().nouns(text)
    nouns = [noun for noun in nouns if noun not in stop_words]
    return [noun for noun in nouns if (len(noun) > 1)]

### 뉴스 기사 불러오기( 증권 네이버 뉴스 -2017,2018,2019,2020)

In [6]:
df1=pd.read_csv('./데이터/result(naver_news)-증권(2017년).csv')
df2=pd.read_csv('./데이터/result(naver_news)-증권(2018년).csv')
df3=pd.read_csv('./데이터/result(naver_news)-증권(2019년).csv')
df4=pd.read_csv('./데이터/result(naver_news)-증권(2020년).csv')

main_df=pd.concat([df1,df2,df3,df4])

In [7]:
main_df.drop('Unnamed: 0', axis=1, inplace=True)

In [8]:
main_df.head()

제목                  작성시간  \
0              [인사] 국민연금공단 기금운용본부  2017.01.01. 오후 10:53   
1         국민연금 기금운용본부 운용전략실장에 이수철   2017.01.01. 오후 9:37   
2       "금리 오르면 사모펀드에 투자기회 늘어날 것"   2017.01.01. 오후 7:45   
3  [마켓인사이트] KB증권, JW바이오 지분 60% 인수   2017.01.01. 오후 7:41   
4      증권사, 이달부터 ELS 헤지자산 구분 관리해야   2017.01.01. 오후 7:39   

                                                  본문  
0  \n\n\n\n\n[한국경제TV 김종학 기자]◇ 전보 △ 운용전략실장 이수철 △ 대...  
1  \n\n\n\n\n\t\n\t국민연금공단 기금운용본부는 1일 공석이던 운용전략실장에...  
2  \n\n\n\n\n미국 사모펀드 운용사 베인캐피털의 존 코너톤 대표과도하게 오른 기...  
3  \n\n\n\n\n통합 법인 출범 첫 작품PEF 결성해 투자…2019년 상장[ 김익...  
4  \n\n\n\n\n\t\n\t[ 안상미 기자  ] 이달부터 증권회사들은 주가연계증권...

In [9]:
del df1, df2, df3, df4

## 뉴스에 값이 없는 행 제거

In [10]:
main_df.isnull().sum()

제목      1
작성시간    6
본문      6
dtype: int64

In [11]:
main_df.dropna(inplace=True)

In [16]:
main_df = main_df.reset_index(drop=True)

In [17]:
main_df

제목                  작성시간  \
0                                [인사] 국민연금공단 기금운용본부  2017.01.01. 오후 10:53   
1                           국민연금 기금운용본부 운용전략실장에 이수철   2017.01.01. 오후 9:37   
2                         "금리 오르면 사모펀드에 투자기회 늘어날 것"   2017.01.01. 오후 7:45   
3                    [마켓인사이트] KB증권, JW바이오 지분 60% 인수   2017.01.01. 오후 7:41   
4                        증권사, 이달부터 ELS 헤지자산 구분 관리해야   2017.01.01. 오후 7:39   
...                                             ...                   ...   
1419312             한국항공우주, 국방과학연구소와 1409억 규모 계약 체결   2020.12.31. 오후 1:37   
1419313      사라지지 않은 코로나19…백신·치료제株, 더 오른다[2021 유망주]   2020.12.31. 오후 1:34   
1419314  동학개미 덕에 두 배 뛴 '반도체'…슈퍼사이클로 더 간다 [2021 유망주]   2020.12.31. 오후 1:34   
1419315                 채권 전문가 69% "내년 채권 금리 보합 전망"   2020.12.31. 오후 1:33   
1419316                                  [인사]KR투자증권   2020.12.31. 오후 1:29   

                                                        본문  
0        \n\n\n\n\n[한국경제TV 김종학 기자]◇ 전보 △ 운용전략실장 이수철 △ 대...  
1        \n\n\n\n\n\t\n\t국민연금공단 기금운용본부는 1일 공석이던 운용전략실장에...  
2        \n\n\n\n\n미국 사모펀드 운용사 베인캐피털의 존 코너톤 대표과도하게 오른 기...  
3        \n\n\n\n\n통합 법인 출범 첫 작품PEF 결성해 투자…2019년 상장[ 김익...  
4        \n\n\n\n\n\t\n\t[ 안상미 기자  ] 이달부터 증권회사들은 주가연계증권...  
...                                                    ...  
1419312  \n\n\n\n\n\t\n\t[헤럴드경제=증권부] 한국항공우주산업은 국방과학연구소와...  
1419313  \n\n\n\n\n진단키트주 오르자 백신주도 상승  치료제 개발 종목에도 관심 쏠려...  
1419314  \n\n\n\n\n개인, 지난해 삼성전자·SK하이닉스 10조원 사들여"삼성전자, 파...  
1419315  \n\n\n\n\n\t\n\t  [파이낸셜뉴스] 대부분의 채권 전문가들은 내년 1월...  
1419316  \n\n\n\n\n\t\n\t [서울=뉴시스]  ◇전무 승진▲대체투자본부 손영호◇이...  

[1419317 rows x 3 columns]

## 토큰화를 위한 전처리

In [19]:
main_df['본문'] = main_df['본문'].apply(text_clean)

### 각 기사를 명사 단어로 토큰화

In [20]:
%%time
main_df['본문키워드'] = main_df['본문'].apply(noun_tokenizer)

CPU times: user 28min 59s, sys: 24min 10s, total: 53min 9s
Wall time: 57min 14s


In [21]:
main_df

제목                  작성시간  \
0                                [인사] 국민연금공단 기금운용본부  2017.01.01. 오후 10:53   
1                           국민연금 기금운용본부 운용전략실장에 이수철   2017.01.01. 오후 9:37   
2                         "금리 오르면 사모펀드에 투자기회 늘어날 것"   2017.01.01. 오후 7:45   
3                    [마켓인사이트] KB증권, JW바이오 지분 60% 인수   2017.01.01. 오후 7:41   
4                        증권사, 이달부터 ELS 헤지자산 구분 관리해야   2017.01.01. 오후 7:39   
...                                             ...                   ...   
1419312             한국항공우주, 국방과학연구소와 1409억 규모 계약 체결   2020.12.31. 오후 1:37   
1419313      사라지지 않은 코로나19…백신·치료제株, 더 오른다[2021 유망주]   2020.12.31. 오후 1:34   
1419314  동학개미 덕에 두 배 뛴 '반도체'…슈퍼사이클로 더 간다 [2021 유망주]   2020.12.31. 오후 1:34   
1419315                 채권 전문가 69% "내년 채권 금리 보합 전망"   2020.12.31. 오후 1:33   
1419316                                  [인사]KR투자증권   2020.12.31. 오후 1:29   

                                                        본문  \
0        한국경제TV김종학기자전보△운용전략실장이수철△대체투자실장김재범발령일년월일김종학기자jh...   
1        국민연금공단기금운용본부는일공석이던운용전략실장에이수철대체투자실장을선임했다김재범기업투자...   
2        미국사모펀드운용사베인캐피털의존코너톤대표과도하게오른기업가치정상복귀할것캐나다구스같은성장...   
3        통합법인출범첫작품PEF결성해투자년상장김익환기자마켓인사이트월일오후시분KB증권이사모펀드...   
4        안상미기자이달부터증권회사들은주가연계증권ELS발행자금으로운용하는자산에꼬리표를붙여야한다...   
...                                                    ...   
1419312  헤럴드경제증권부한국항공우주산업은국방과학연구소와LinkK완성형항공기체계통합계약을체결했...   
1419313  진단키트주오르자백신주도상승치료제개발종목에도관심쏠려셀트리온진원생명과학제넥신등개발현황따...   
1419314  개인지난해삼성전자SK하이닉스조원사들여삼성전자파운드리실적추가상승전망SK하이닉스DRAM...   
1419315  파이낸셜뉴스대부분의채권전문가들은내년월국내채권금리가보합세를보일것으로전망했다일금융투자협...   
1419316  서울뉴시스전무승진▲대체투자본부손영호이사승진▲매크로투자팀노재호▲매크로투자팀김지환네이버...   

                                                     본문키워드  
0        [한국, 경제, 김종학, 전보, 운용, 전략, 실장, 수철, 대체, 투자, 실장, ...  
1        [국민, 연금, 공단, 기금, 운용, 본부, 공석, 운용, 전략, 실장, 이수철, ...  
2        [미국, 사모, 펀드, 운용, 캐피털, 코너, 대표, 과도, 기업, 가치, 정상, ...  
3        [통합, 법인, 출범, 작품, 결성, 투자, 상장, 김익환, 마켓, 인사이트, 월일...  
4        [안상미, 이달, 회사, 주가, 연계증, 발행, 자금, 운용, 자산, 꼬리표, 위험...  
...                                                    ...  
1419312  [헤럴드, 경제, 한국, 항공우주, 산업, 국방과학연구소, 완성, 항공기, 체계, ...  
1419313  [진단, 키트, 백신, 주도, 상승, 치료제, 개발, 종목, 관심, 트리, 진원, ...  
1419314  [개인, 지난해, 삼성전자, 하이닉스, 조원, 삼성전자, 파운드리, 실적, 상승, ...  
1419315  [파이낸셜, 대부분, 채권, 전문가, 내년, 국내, 채권, 금리, 보합세, 전망, ...  
1419316  [서울, 뉴시스, 전무, 승진, 대체, 투자, 본부, 손영호, 이사, 승진, 매크로...  

[1419317 rows x 4 columns]

In [22]:
main_df.to_csv('./main_df')

### Gensim의 Phrases 모듈을 이용하여 유사 토큰을 묶어줌

In [23]:
%%time
bigram=Phrases(sentences=main_df['본문키워드'], min_count=30, threshold=100, delimiter='')

CPU times: user 2min 51s, sys: 15.7 s, total: 3min 6s
Wall time: 3min 25s


In [24]:
bigram.vocab

{'한국': 1207969,
 '경제': 2208050,
 '한국경제': 784919,
 '김종학': 502,
 '경제김종학': 180,
 '전보': 1385,
 '김종학전보': 2,
 '운용': 231114,
 '전보운용': 4,
 '전략': 120595,
 '운용전략': 2894,
 '실장': 11427,
 '전략실장': 496,
 '수철': 23,
 '실장수철': 1,
 '대체': 38338,
 '수철대체': 1,
 '투자': 2192372,
 '대체투자': 11420,
 '투자실장': 372,
 '김재범': 62,
 '실장김재범': 10,
 '발령': 1552,
 '김재범발령': 1,
 '월일': 459492,
 '발령월일': 14,
 '월일김종학': 3,
 '김종학한국': 83,
 '방주': 408,
 '경제방주': 130,
 '여기': 48621,
 '방주여기': 130,
 '영상': 122317,
 '여기영상': 133,
 '영상한국': 106,
 '전재': 722599,
 '경제전재': 492899,
 '국민': 139246,
 '연금': 95252,
 '국민연금': 44276,
 '공단': 14141,
 '연금공단': 5786,
 '기금': 26893,
 '공단기금': 736,
 '기금운용': 9261,
 '본부': 75587,
 '운용본부': 9287,
 '공석': 1200,
 '본부공석': 47,
 '공석운용': 11,
 '이수철': 136,
 '실장이수철': 2,
 '이수철대체': 3,
 '선임': 44659,
 '실장선임': 77,
 '선임김재범': 1,
 '기업': 1041231,
 '김재범기업': 2,
 '기업투자': 17578,
 '팀장': 27533,
 '투자팀장': 435,
 '승진': 5638,
 '팀장승진': 117,
 '후임': 1486,
 '승진후임': 2,
 '후임대체': 3,
 '이번': 382238,
 '실장이번': 193,
 '인사': 19479,
 '이번인사': 676,
 '양영식': 43,
 '인사양영식': 2

### 문서 분석을 위해서 토큰들을 bigram화 하여 리스트에 담아줌

* 각 기사들의 token들을 bigram의 key로 주면 토큰을 bigram으로 만들어줌 

In [25]:
%%time
bi_token_list=[]

for tokens in main_df['본문키워드']:
    bi_token_list.append([token for token in bigram[tokens] if token not in stop_words])

CPU times: user 5min 18s, sys: 1min 2s, total: 6min 20s
Wall time: 8min 58s


### Doc2Vec 적용을 위해 분석에 적합한 형식으로 바꿔줌

* Doc2Vec의 경우 분석을 위한 토큰들이 단순히 리스트가 아닌 TaggeedDocument 형식에 담겨야함
* TaggedDocument는 (토큰리스트, 태그)와 같은 형식으로 구성되어 있으며, 태크에는 해당 문서를 대표하는 단어를 넣어줌. 

In [26]:
documents=[TaggedDocument(bi_token_list[i],[i]) for i in range(len(bi_token_list))]

## Doc2Vec 모델 생성

* vector_size는 뉴스를 몇개의 feature를 사용하여 vector로 만들 것인지(여기서는 100개의 feature로 구분하도록 지정)
* window는 뉴스 내의 토큰을 분석할 때 특정 토큰 주위 몇 개의 토큰을 함께 분석할 것인지(여기서는 토큰 앞 뒤 각각 5개의 토큰을 함께 분석)
* min_count는 최소 몇 개의 뉴스에 나온 토큰어들을 분석에 포함시킬 것인지(10개 미만의 뉴스에만 있는 토큰 분석에서 제외)
* epochs는 모델 생성시 학습하는 횟수 

In [28]:
%%time
model=Doc2Vec(documents, vector_size=100, window=5, workers=4, min_count=10, epochs=40)

CPU times: user 3h 18min 19s, sys: 15min 22s, total: 3h 33min 42s
Wall time: 1h 38min 19s


In [29]:
# 모델의 토큰빌드
model.build_vocab(documents)

In [31]:
%%time
# 모델의 학습시작
model.train(documents, epochs=40, total_examples=model.corpus_count)

CPU times: user 3h 21min 17s, sys: 15min 51s, total: 3h 37min 9s
Wall time: 1h 29min 12s


In [32]:
model.save('./doc2vec_category.model')

In [33]:
# 전체기사수
model.dv.vectors.shape

(1419317, 100)

In [34]:
pd.DataFrame(model.dv.vectors)

0         1         2         3         4         5         6   \
0       -0.169655 -0.459485  0.739331 -0.975861 -0.300233  0.091107  0.527048   
1        0.004984  1.207775  0.533389 -2.219108 -0.881941 -0.489143 -0.003795   
2       -1.615873 -4.859083  0.441344  1.223015 -2.114782  0.327292 -0.581605   
3        0.293425  0.123222  1.930987 -2.561589 -0.259596 -1.925607 -0.428440   
4        2.348254 -0.170722  0.428996 -0.645139 -2.739914 -3.418238  1.667897   
...           ...       ...       ...       ...       ...       ...       ...   
1419312  0.343322 -0.353120  0.718962 -0.828677 -0.278351 -0.150899  0.264459   
1419313 -3.228831  0.091273 -0.937584 -2.545386  1.478824 -0.049323  0.704190   
1419314 -0.591351  0.197083  0.267623 -1.524850  0.140706  0.162136 -1.655023   
1419315  0.033289 -0.613696 -1.018771  0.100819  0.626553 -0.081900 -0.758080   
1419316 -0.299854 -0.274174  0.210055 -0.500832 -0.770198  0.469197 -0.058171   

               7         8         9   ...        90        91        92  \
0       -0.633131  0.186986  0.092903  ... -0.164301 -0.123287  1.093875   
1       -0.525700  0.197927  0.902119  ...  1.439796 -0.614903  1.438943   
2       -1.185222  0.088461 -0.343948  ...  1.306044  0.785357  2.644317   
3        1.780559  0.994884  2.237341  ... -1.888568  0.783017  2.418608   
4        3.889777 -2.541707 -1.169806  ... -1.923474 -3.126075  0.171746   
...           ...       ...       ...  ...       ...       ...       ...   
1419312  0.291850 -0.179297 -0.066245  ...  0.215770  0.447943 -0.316626   
1419313  0.617130 -0.070736  3.081007  ...  1.885930  0.464880  0.633910   
1419314  3.583301  0.094606 -2.834288  ... -0.486369 -1.572897 -1.833151   
1419315 -2.987689  1.219467  0.145450  ...  0.907558 -0.030317 -0.139035   
1419316  0.359926  0.071572 -0.820398  ...  0.270272 -0.688870  1.145066   

               93        94        95        96        97        98        99  
0        0.254903 -1.771094 -1.024852  0.511426 -0.143863  0.891536  0.069893  
1       -0.301139  0.112022  0.318524  1.021725  0.273799  0.815545 -0.602558  
2       -2.004651  0.194567  4.727800  0.961755  0.726398  1.801158  1.913976  
3       -0.051383 -0.461607 -0.810304 -2.529619 -1.987902  2.390100 -2.505969  
4        0.171543  0.130286 -1.168251  1.365541  3.158936  0.912573  0.703124  
...           ...       ...       ...       ...       ...       ...       ...  
1419312  0.207669 -0.979945 -1.032540 -0.311230 -0.053037 -0.190989  0.872487  
1419313  0.778977 -0.489366  1.496024  3.173354  2.187582 -0.271706 -1.361573  
1419314  0.255308  0.447903 -0.153730  2.383433 -0.022837 -0.578241  0.211397  
1419315 -0.282165 -1.389319  0.257530  1.325590 -3.148491  1.092502  2.058374  
1419316 -0.713666 -0.710612 -0.820462 -0.138570  0.172490  0.647190  0.510374  

[1419317 rows x 100 columns]

### 생성된 Doc2Vec 모델 테스트
* 임의의 뉴스 기사 혹은 단어를 통해서, 주어진 단어나 기사와 유사한 기사를 선별

In [35]:
# 모델 테스트를 위한 문장 또는 단어
text="""삼성전자"""

In [37]:
# 학습된 모델을 바탕으로 token을 vector로 변환
vectors=model.infer_vector(text)

In [38]:
vectors

array([ 0.0814442 ,  0.04815671,  0.11152497, -0.18982804, -0.13580318,
        0.03597274, -0.12567207, -0.04818019, -0.10721406, -0.08198759,
        0.20760581, -0.01144214,  0.12518537,  0.01782097, -0.00555768,
       -0.02551647,  0.12460998,  0.10515765, -0.08334742, -0.10211912,
        0.223848  , -0.06406574, -0.05788188, -0.06366919, -0.16671236,
        0.03531377, -0.19770512, -0.15737097, -0.00095933, -0.03738008,
       -0.10521876, -0.04037554, -0.17655781,  0.01175783,  0.02387102,
       -0.04294837,  0.08819685,  0.09256206, -0.13952008, -0.1273672 ,
       -0.12464681,  0.14533982,  0.11273479, -0.03876241,  0.03681689,
        0.1788992 ,  0.07476626, -0.1655962 , -0.13397376,  0.15264253,
        0.0084673 ,  0.21546784,  0.09692518, -0.06417481,  0.1484902 ,
       -0.13627772,  0.11813674, -0.06634613, -0.10037069,  0.11331471,
       -0.0651449 , -0.0139699 ,  0.06176943,  0.06096971,  0.2411155 ,
       -0.10412671, -0.00581036, -0.0604187 ,  0.05457265,  0.21

In [66]:
# most_similar 메소드를 이용해서, 주어진 벡터와 유사한 기사들을 가져옴.
# topn=100을 통해 유사한 100개의 기사를 가자ㅕ옴
model.dv.most_similar(vectors, topn=1419317)

# 각 값은 (기사의 인덱스, cosine_similarity값)

[(1062621, 0.8716700673103333),
 (977598, 0.8694313764572144),
 (740461, 0.8641110062599182),
 (977589, 0.8639147281646729),
 (740459, 0.8568779826164246),
 (973989, 0.8548345565795898),
 (994582, 0.851881742477417),
 (787379, 0.848913848400116),
 (257447, 0.8480136394500732),
 (565573, 0.8452557921409607),
 (675401, 0.8442733287811279),
 (918622, 0.8398178815841675),
 (700582, 0.8397209644317627),
 (347649, 0.8383817076683044),
 (181866, 0.8381497859954834),
 (641981, 0.8376025557518005),
 (288503, 0.8356385231018066),
 (608162, 0.8334575891494751),
 (448368, 0.8334185481071472),
 (932725, 0.8314583897590637),
 (546721, 0.8305817246437073),
 (787383, 0.8297384977340698),
 (44881, 0.8296905159950256),
 (6216, 0.8293806314468384),
 (350758, 0.8292546272277832),
 (257440, 0.8290069103240967),
 (465744, 0.8284193873405457),
 (587111, 0.827751100063324),
 (819780, 0.8275641202926636),
 (822864, 0.8275235295295715),
 (1034723, 0.8274646401405334),
 (692265, 0.8272207379341125),
 (655915, 0.

In [67]:
score=0.5

# 유사한 기사 목록을 뽑아오기.
# 위의 out 값에서 cosine_similarity 값이 socre(여기서는 0.5)보다 클 경우 index를 리스트에 담음. 
sim_list=[i[0] for i in model.dv.most_similar(vectors, topn=1419317) if i[1] >=score]

In [68]:
# 유사 기사의 인덱스를 기사 DataFrame에 넣어서 기사 내용 가져오기.
sub_df=main_df.loc[sim_list]

In [69]:
# 논조 분류를 위해서 index를 리셋해줌
sub_df = sub_df.reset_index(drop=True)

In [70]:
sub_df

제목                 작성시간  \
0                삼성전자 1분기 영업익 6조2천333억원…10분기만에 최저치(1보)  2019.04.30. 오전 8:42   
1               삼성전자 1분기 영업익 6.2조원…전분기 대비 42.6% 감소(1보)  2019.04.05. 오전 8:36   
2              삼성전자 4분기 영업익 10.8조원…전분기 대비 38.5% 감소(1보)  2019.01.08. 오전 8:34   
3               삼성전자 영업익 10분기만에 최저치…증권사 전망치 평균 하회 (속보)  2019.04.05. 오전 8:37   
4                 삼성전자 영업익 7분기만에 최저치…전망치 하회 '어닝쇼크'(속보)  2019.01.08. 오전 8:34   
...                                                ...                  ...   
9658                   코스피, 2거래일째 '숨고르기'…기관 3000억원 순매도  2017.06.07. 오후 3:58   
9659  [한경로보뉴스] '테스나' 5% 이상 상승, 외국계 증권사 창구의 거래비중 14% 수준  2018.05.29. 오후 3:14   
9660                                     소폭 하락 마감한 코스피  2020.07.23. 오후 3:47   
9661                              동진쎄미켐, 1주당 60원 현금 배당  2017.03.08. 오후 3:28   
9662                  [생생코스닥]아이앤씨, 한국전력과 28.9억 규모 공급계약  2017.09.20. 오후 3:04   

                                                     본문  \
0     삼성전자CG연합뉴스TV제공네이버홈에서연합뉴스채널구독하기뭐하고놀까흥쇼미더뉴스오늘많이본...   
1     삼성전자CG연합뉴스TV제공네이버홈에서연합뉴스채널구독하기뭐하고놀까흥쇼미더뉴스오늘많이본...   
2     삼성전자CG연합뉴스TV제공연합뉴스채널구독하고스타벅스커피드세요뭐하고놀까흥쇼미더뉴스오늘...   
3     삼성전자CG연합뉴스TV제공네이버홈에서연합뉴스채널구독하기뭐하고놀까흥쇼미더뉴스오늘많이본...   
4     삼성전자CG연합뉴스TV제공연합뉴스채널구독하고스타벅스커피드세요뭐하고놀까흥쇼미더뉴스오늘...   
...                                                 ...   
9658  아시아경제박나영기자코스피가거래일연속하락마감했다기관이나홀로억원을팔아치운영향이다일코스피...   
9659  일오후시분현재테스나는전거래일보다오른원에거래되고있다주가의흐름은원으로개장해서장중에밀리지...   
9660  서울연합뉴스최재구기자일서울명동하나은행본점에서직원이코스피전광판앞을지나고있다이날코스피는...   
9661  머니투데이이원광기자동진쎄미켐은보통주주당원의현금배당을결정했다고일공시했다배당금총액은억만...   
9662  헤럴드경제증권팀아이앤씨테크놀로지는한국전력공사와저압AMI용PLC모뎀및DCU공급계약을체...   

                                                  본문키워드  
0               [삼성전자, 연합뉴스, 제공, 네이버, 연합뉴스, 채널, 구독, 영상]  
1               [삼성전자, 연합뉴스, 제공, 네이버, 연합뉴스, 채널, 구독, 영상]  
2          [삼성전자, 연합뉴스, 제공, 연합, 채널, 구독, 스타, 벅스, 커피, 영상]  
3               [삼성전자, 연합뉴스, 제공, 네이버, 연합뉴스, 채널, 구독, 영상]  
4          [삼성전자, 연합뉴스, 제공, 연합, 채널, 구독, 스타, 벅스, 커피, 영상]  
...                                                 ...  
9658  [아시아, 경제, 나영, 코스피, 거래일, 연속, 하락, 마감, 기관, 영향, 다일...  
9659  [오후, 시분, 테스, 거래일, 거래, 주가, 흐름, 개장, 장중, 상승, 서원, ...  
9660  [서울, 연합뉴스, 최재구, 서울, 명동, 하나, 은행, 본점, 직원, 코스피, 전...  
9661  [머니, 투데이, 이원광, 동진, 쎄미, 보통, 주주, 당원, 현금, 배당, 결정,...  
9662  [헤럴드, 경제, 아이, 테크놀로지, 한국전력공사, 저압, 모뎀, 공급, 계약, 체...  

[9663 rows x 4 columns]

## 유사 기사들을 바탕으로 삼성전자 관련기사 분류


### TfidfVectorizer를 이용해 각 뉴스기사에 동일하게 존재하는 토큰 제거

In [74]:
# TfidfVectorizer를 이용해 각 뉴스기사에 존재하는 토큰 갯수 확인
# min_df를 전체 뉴스기사 수의 절반을 주어, 많은 기사들에 존재하는 토큰들만 확인
tfidf_vect = TfidfVectorizer(tokenizer=noun_tokenizer, min_df = round(len(sub_df)/3))
matrix = tfidf_vect.fit_transform(sub_df['본문']).toarray()

In [75]:
matrix_df = pd.DataFrame(matrix, columns=tfidf_vect.get_feature_names())

In [76]:
matrix_df

거래        경제        공시       기간       기사        대비        분석  \
0     0.000000  0.000000  0.000000  0.00000  0.00000  0.000000  0.000000   
1     0.000000  0.000000  0.000000  0.00000  0.00000  0.000000  0.000000   
2     0.000000  0.000000  0.000000  0.00000  0.00000  0.000000  0.000000   
3     0.000000  0.000000  0.000000  0.00000  0.00000  0.000000  0.000000   
4     0.000000  0.000000  0.000000  0.00000  0.00000  0.000000  0.000000   
...        ...       ...       ...      ...      ...       ...       ...   
9658  0.373368  0.811314  0.000000  0.00000  0.00000  0.315721  0.000000   
9659  0.604686  0.291992  0.000000  0.00000  0.54088  0.000000  0.402011   
9660  0.000000  0.000000  0.000000  0.00000  0.00000  0.000000  0.000000   
9661  0.000000  0.000000  0.549479  0.00000  0.00000  0.000000  0.000000   
9662  0.000000  0.728276  0.338860  0.33707  0.00000  0.283407  0.000000   

          삼성전자        월일        작성        저작        전재        주가        투자  
0     1.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
1     1.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
2     1.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
3     1.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
4     1.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
...        ...       ...       ...       ...       ...       ...       ...  
9658  0.320447  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
9659  0.000000  0.000000  0.196691  0.000000  0.138735  0.192345  0.000000  
9660  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  
9661  0.000000  0.000000  0.000000  0.539069  0.374070  0.000000  0.517252  
9662  0.000000  0.328079  0.000000  0.000000  0.230686  0.000000  0.000000  

[9663 rows x 14 columns]

In [77]:
# 0.0이 아닌 값들(해당 뉴스기사에 존재하는 토큰)의 숫자를 세고, 전체 뉴스기사 수 40% 이상에서 관찰된 토큰은 제거
# 예를들면 50개의 뉴스기사 중 20개(40%)이상의 문서에서 동일하게 발견되는 토큰 제거
(matrix_df !=0.0).sum()

거래      3312
경제      5862
공시      3237
기간      3273
기사      4121
대비      4560
분석      3331
삼성전자    4442
월일      3460
작성      3482
저작      3368
전재      6316
주가      3641
투자      3660
dtype: int64

In [86]:
# 전체의 40% 이상에서 발견되는 토큰들을 deprecated_words 리스트에 담아줌
deprecated_words=(matrix_df !=0.0).sum()[((matrix_df !=0.0).sum() >= round(len(sub_df)*0.4))].index

In [87]:
deprecated_words

Index(['경제', '기사', '대비', '삼성전자', '전재'], dtype='object')

In [88]:
# deprecated_words를 적용하기 위한 함수
def make_small_token(temp_list):
    return [token for token in temp_list if token not in deprecated_words]

# apply메소드를 이용해, 각 뉴스기사의 토큰들에서 제거
sub_df['small_token']=sub_df['본문키워드'].apply(make_small_token)

In [89]:
# ['경제', '기사', '대비', '삼성전자', '전재']토큰만이 제거됨.
sub_df

제목                 작성시간  \
0                삼성전자 1분기 영업익 6조2천333억원…10분기만에 최저치(1보)  2019.04.30. 오전 8:42   
1               삼성전자 1분기 영업익 6.2조원…전분기 대비 42.6% 감소(1보)  2019.04.05. 오전 8:36   
2              삼성전자 4분기 영업익 10.8조원…전분기 대비 38.5% 감소(1보)  2019.01.08. 오전 8:34   
3               삼성전자 영업익 10분기만에 최저치…증권사 전망치 평균 하회 (속보)  2019.04.05. 오전 8:37   
4                 삼성전자 영업익 7분기만에 최저치…전망치 하회 '어닝쇼크'(속보)  2019.01.08. 오전 8:34   
...                                                ...                  ...   
9658                   코스피, 2거래일째 '숨고르기'…기관 3000억원 순매도  2017.06.07. 오후 3:58   
9659  [한경로보뉴스] '테스나' 5% 이상 상승, 외국계 증권사 창구의 거래비중 14% 수준  2018.05.29. 오후 3:14   
9660                                     소폭 하락 마감한 코스피  2020.07.23. 오후 3:47   
9661                              동진쎄미켐, 1주당 60원 현금 배당  2017.03.08. 오후 3:28   
9662                  [생생코스닥]아이앤씨, 한국전력과 28.9억 규모 공급계약  2017.09.20. 오후 3:04   

                                                     본문  \
0     삼성전자CG연합뉴스TV제공네이버홈에서연합뉴스채널구독하기뭐하고놀까흥쇼미더뉴스오늘많이본...   
1     삼성전자CG연합뉴스TV제공네이버홈에서연합뉴스채널구독하기뭐하고놀까흥쇼미더뉴스오늘많이본...   
2     삼성전자CG연합뉴스TV제공연합뉴스채널구독하고스타벅스커피드세요뭐하고놀까흥쇼미더뉴스오늘...   
3     삼성전자CG연합뉴스TV제공네이버홈에서연합뉴스채널구독하기뭐하고놀까흥쇼미더뉴스오늘많이본...   
4     삼성전자CG연합뉴스TV제공연합뉴스채널구독하고스타벅스커피드세요뭐하고놀까흥쇼미더뉴스오늘...   
...                                                 ...   
9658  아시아경제박나영기자코스피가거래일연속하락마감했다기관이나홀로억원을팔아치운영향이다일코스피...   
9659  일오후시분현재테스나는전거래일보다오른원에거래되고있다주가의흐름은원으로개장해서장중에밀리지...   
9660  서울연합뉴스최재구기자일서울명동하나은행본점에서직원이코스피전광판앞을지나고있다이날코스피는...   
9661  머니투데이이원광기자동진쎄미켐은보통주주당원의현금배당을결정했다고일공시했다배당금총액은억만...   
9662  헤럴드경제증권팀아이앤씨테크놀로지는한국전력공사와저압AMI용PLC모뎀및DCU공급계약을체...   

                                                  본문키워드  \
0               [삼성전자, 연합뉴스, 제공, 네이버, 연합뉴스, 채널, 구독, 영상]   
1               [삼성전자, 연합뉴스, 제공, 네이버, 연합뉴스, 채널, 구독, 영상]   
2          [삼성전자, 연합뉴스, 제공, 연합, 채널, 구독, 스타, 벅스, 커피, 영상]   
3               [삼성전자, 연합뉴스, 제공, 네이버, 연합뉴스, 채널, 구독, 영상]   
4          [삼성전자, 연합뉴스, 제공, 연합, 채널, 구독, 스타, 벅스, 커피, 영상]   
...                                                 ...   
9658  [아시아, 경제, 나영, 코스피, 거래일, 연속, 하락, 마감, 기관, 영향, 다일...   
9659  [오후, 시분, 테스, 거래일, 거래, 주가, 흐름, 개장, 장중, 상승, 서원, ...   
9660  [서울, 연합뉴스, 최재구, 서울, 명동, 하나, 은행, 본점, 직원, 코스피, 전...   
9661  [머니, 투데이, 이원광, 동진, 쎄미, 보통, 주주, 당원, 현금, 배당, 결정,...   
9662  [헤럴드, 경제, 아이, 테크놀로지, 한국전력공사, 저압, 모뎀, 공급, 계약, 체...   

                                            small_token  
0                     [연합뉴스, 제공, 네이버, 연합뉴스, 채널, 구독, 영상]  
1                     [연합뉴스, 제공, 네이버, 연합뉴스, 채널, 구독, 영상]  
2                [연합뉴스, 제공, 연합, 채널, 구독, 스타, 벅스, 커피, 영상]  
3                     [연합뉴스, 제공, 네이버, 연합뉴스, 채널, 구독, 영상]  
4                [연합뉴스, 제공, 연합, 채널, 구독, 스타, 벅스, 커피, 영상]  
...                                                 ...  
9658  [아시아, 나영, 코스피, 거래일, 연속, 하락, 마감, 기관, 영향, 다일, 코스...  
9659  [오후, 시분, 테스, 거래일, 거래, 주가, 흐름, 개장, 장중, 상승, 서원, ...  
9660  [서울, 연합뉴스, 최재구, 서울, 명동, 하나, 은행, 본점, 직원, 코스피, 전...  
9661  [머니, 투데이, 이원광, 동진, 쎄미, 보통, 주주, 당원, 현금, 배당, 결정,...  
9662  [헤럴드, 아이, 테크놀로지, 한국전력공사, 저압, 모뎀, 공급, 계약, 체결, 공...  

[9663 rows x 5 columns]

### small_token(너무 많이 존재하는 토큰이 삭제된 토큰목록)과 KMeans을 이용해서 Clustering

In [92]:
# 각 기사를 벡터화 하기 위해서 TfidfVectorizer 이용
# 문자열이 아닌 토큰으로 학습하기 위해서 tokenizer를 지정하지 않음(띄어쓰기를 바탕으로 토큰을 자동 구분)
split_tfidf=TfidfVectorizer()

# TfidfVectorizer의 경우 문장(String 타입의 텍스트)이 들어가야하나,
# small_token(토큰이 들어가있는 리스트)를 통해 분석하기 위해서, ''.join(list)메소드를 이용해서 리스트를 문자열로 바꿔줌
split_matrix=split_tfidf.fit_transform(sub_df['small_token'].apply(lambda x: ' '.join(x))).toarray()

In [94]:
split_matrix_df=pd.DataFrame(split_matrix, columns=split_tfidf.get_feature_names())


In [102]:
split_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [95]:
split_matrix_df

가개   가격  가격제   가년   가늠   가능   가닉   가대   가동   가량  ...   흥국   흥부   흥아  \
0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
3     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
4     0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
...   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
9658  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
9659  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
9660  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
9661  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   
9662  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...  0.0  0.0  0.0   

       흥원   희기   희망   희비   희연   힐링   힘스  
0     0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1     0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2     0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3     0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4     0.0  0.0  0.0  0.0  0.0  0.0  0.0  
...   ...  ...  ...  ...  ...  ...  ...  
9658  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
9659  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
9660  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
9661  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
9662  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[9663 rows x 6313 columns]

### KMeans를 이용해 군집화

In [96]:
# 기사를 분류하기 위해 n_clusters는 3으로 지정
kmm=KMeans(n_clusters=3)
kmm_preds=kmm.fit_predict(split_matrix)

In [97]:
pd.Series(kmm_preds).value_counts()

0    6304
1    2347
2    1012
dtype: int64

In [98]:
final_df=sub_df[['제목','본문']]
final_df['kmm_cl']=kmm_preds

/Users/eunjeongs/opt/anaconda3/envs/Tensorflow_Test/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [99]:
# 1. <영업이익>에 대한 분류
final_df[final_df.kmm_cl==0]

제목  \
0                삼성전자 1분기 영업익 6조2천333억원…10분기만에 최저치(1보)   
1               삼성전자 1분기 영업익 6.2조원…전분기 대비 42.6% 감소(1보)   
2              삼성전자 4분기 영업익 10.8조원…전분기 대비 38.5% 감소(1보)   
3               삼성전자 영업익 10분기만에 최저치…증권사 전망치 평균 하회 (속보)   
4                 삼성전자 영업익 7분기만에 최저치…전망치 하회 '어닝쇼크'(속보)   
...                                                ...   
9658                   코스피, 2거래일째 '숨고르기'…기관 3000억원 순매도   
9659  [한경로보뉴스] '테스나' 5% 이상 상승, 외국계 증권사 창구의 거래비중 14% 수준   
9660                                     소폭 하락 마감한 코스피   
9661                              동진쎄미켐, 1주당 60원 현금 배당   
9662                  [생생코스닥]아이앤씨, 한국전력과 28.9억 규모 공급계약   

                                                     본문  kmm_cl  
0     삼성전자CG연합뉴스TV제공네이버홈에서연합뉴스채널구독하기뭐하고놀까흥쇼미더뉴스오늘많이본...       0  
1     삼성전자CG연합뉴스TV제공네이버홈에서연합뉴스채널구독하기뭐하고놀까흥쇼미더뉴스오늘많이본...       0  
2     삼성전자CG연합뉴스TV제공연합뉴스채널구독하고스타벅스커피드세요뭐하고놀까흥쇼미더뉴스오늘...       0  
3     삼성전자CG연합뉴스TV제공네이버홈에서연합뉴스채널구독하기뭐하고놀까흥쇼미더뉴스오늘많이본...       0  
4     삼성전자CG연합뉴스TV제공연합뉴스채널구독하고스타벅스커피드세요뭐하고놀까흥쇼미더뉴스오늘...       0  
...                                                 ...     ...  
9658  아시아경제박나영기자코스피가거래일연속하락마감했다기관이나홀로억원을팔아치운영향이다일코스피...       0  
9659  일오후시분현재테스나는전거래일보다오른원에거래되고있다주가의흐름은원으로개장해서장중에밀리지...       0  
9660  서울연합뉴스최재구기자일서울명동하나은행본점에서직원이코스피전광판앞을지나고있다이날코스피는...       0  
9661  머니투데이이원광기자동진쎄미켐은보통주주당원의현금배당을결정했다고일공시했다배당금총액은억만...       0  
9662  헤럴드경제증권팀아이앤씨테크놀로지는한국전력공사와저압AMI용PLC모뎀및DCU공급계약을체...       0  

[6304 rows x 3 columns]

In [100]:
# 2. <주식가격>에 대한 분류
final_df[final_df.kmm_cl==1]

제목  \
26       <유>삼성전자, 장중 신저가 기록.. 51,800→51,700(▼100)   
32        <유>삼성전자, 장중 신저가 기록.. 40,550→40,500(▼50)   
37       <유>삼성전자, 장중 신저가 기록.. 40,400→40,300(▼100)   
46       <유>삼성전자, 장중 신저가 기록.. 50,900→50,700(▼200)   
55                삼성전자, 전일 대비 약 3% 상승한 2,308,000원   
...                                           ...   
9643                성우테크론, 전일 대비 약 -3% 하락한 4,400원   
9646       SK하이닉스, 800원(-0.96%) 내린 82,200원에 거래 시작   
9649      (코)윈팩, 전일 대비 7.42% 상승.. 일일회전율은 3.17% 기록   
9650                   엑시콘, 전일 대비 약 3% 하락한 5,880원   
9652  <코>빛샘전자, 전일 대비 10.15% 상승.. 일일회전율은 18.96% 기록   

                                                     본문  kmm_cl  
26    서울경제오전시분현재삼성전자가내린원▼을기록하며주신저가를원경신했다기존주최저가는년월일기록...       1  
32    서울경제오전시분현재삼성전자가내린원▼을기록하며주신저가를원경신했다기존주최저가는년월일기록...       1  
37    서울경제오전시분현재삼성전자가내린원▼을기록하며주신저가를원경신했다기존주최저가는년월일기록...       1  
46    서울경제오전시분현재삼성전자가내린원▼을기록하며주신저가를원경신했다기존주최저가는년월일기록...       1  
55    한국경제TV라이온봇기자삼성전자는월일시분전일대비약상승한원에거래되고있다삼성전자는코스피상...       1  
...                                                 ...     ...  
9643  한국경제TV라이온봇기자그림최근개월간주가추세및기간별수익률성우테크론은월일시분전일대비약하...       1  
9646  한국경제TV라이온봇기자월일SK하이닉스는전일원대비원내린원에거래를시작했다라이온봇기자한국...       1  
9649  서울경제윈팩은일오후시분현재오른원에거래되고있으며매도매수의매수우위를기록하고있다거래대금은...       1  
9650  한국경제TV라이온봇기자그림최근개월간주가추세및기간별수익률엑시콘은월일시분전일대비약하락한...       1  
9652  서울경제빛샘전자는일오전시분현재오른원에거래되고있으며매도매수의매수우위를기록하고있다거래대...       1  

[2347 rows x 3 columns]

In [101]:
# 3. <증권회사에서 바라보는 투자견해>에 대한 분류
final_df[final_df.kmm_cl==2]

제목  \
10      [ET투자뉴스]삼성전자, "따라갈 수 없다…" BUY(유지)-NH투자증권   
19    [ET투자뉴스]삼성전자, "나쁘기도 하지만 좋…" BUY(유지)-미래에셋대우   
24          [ET투자뉴스]삼성전자, "好事多魔…" 매수(유지)-IBK투자증권   
40       [ET투자뉴스]삼성전자, "코리아 디스카운트 …" 매수(유지)-KB증권   
48             [ET투자뉴스]삼성전자, "숨고르기…" BUY-KTB투자증권   
...                                          ...   
9586     [ET투자뉴스]삼성생명, "손해율 개선 및 삼…" 매수(유지)-SK증권   
9607   [로봇뉴스]삼성전기, "2017년, 위를 …" 매수(유지)-이베스트투자증권   
9621  [ET투자뉴스]현대모비스, "흔들림 없이 간다…" BUY(유지)-신한금융투자   
9644     [ET투자뉴스]삼성전자, "예상보다 더 부진한…" 매수(유지)-신영증권   
9645   [ET투자뉴스]삼성전기, "MLCC가 이끄는 …" 매수(유지)-유진투자증권   

                                                     본문  kmm_cl  
10    NH투자증권에서일삼성전자에대해따라갈수없다라며투자의견을BUY유지로제시하였고아울러목표주...       2  
19    미래에셋대우에서일삼성전자에대해나쁘기도하지만좋기도하다라며투자의견을BUY유지로제시하였고...       2  
24    IBK투자증권에서일삼성전자에대해라며투자의견을매수유지로제시하였고아울러목표주가로는원을내...       2  
40    KB증권에서일삼성전자에대해코리아디스카운트해소기대라며투자의견을매수유지로제시하였고아울러...       2  
48    KTB투자증권에서일삼성전자에대해숨고르기라며투자의견을BUY로제시하였고아울러목표주가로는...       2  
...                                                 ...     ...  
9586  SK증권에서일삼성생명에대해손해율개선및삼성전자중간배당으로이익증가기대라며투자의견을매수유...       2  
9607  이베스트투자증권에서일삼성전기에대해년위를보자라며투자의견을매수유지로제시하였고아울러목표주...       2  
9621  신한금융투자에서일현대모비스에대해흔들림없이간다라며투자의견을BUY유지로제시하였고아울러목...       2  
9644  신영증권에서일삼성전자에대해예상보다더부진한반도체라며투자의견을매수유지로제시하였고아울러목...       2  
9645  유진투자증권에서일삼성전기에대해MLCC가이끄는호실적라며투자의견을매수유지로제시하였다아울...       2  

[1012 rows x 3 columns]